This is the base AlexNet CNN code then I will list out the following modiciations I made to it in seperate code sections.

In [ ]:
# Install & import necessary libraries
!pip install patool==1.12
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import patoolib
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Extract the zip file from Google Drive
 patoolib.extract_archive("/content/drive/MyDrive/Hood College/archive.zip", outdir="/content/extracted_images") if the drive is not connected to the program uncomment this line and specify the dir to the zip file or dataset

# define the directory containing the dataset
data_dir = '/content/extracted_images/leapGestRecog'

#load the dataset and split it into training and validation sets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,  # reserve 30% for validation
    subset="training",
    seed=123,
    image_size=(227, 227),  # resize images to 227x227 pixels (AlexNet input size)
    batch_size=32,
    label_mode='categorical'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    seed=123,
    image_size=(227, 227),
    batch_size=32,
    label_mode='categorical'
)

# Display class names to verify labels
class_names = train_dataset.class_names
print("Class Names:", class_names)

# Build the AlexNet model architecture
def build_alexnet(input_shape=(227, 227, 3), num_classes=len(class_names)):
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),

        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),

        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Create the AlexNet model
alexnet_model = build_alexnet()

# Compile the model
alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
alexnet_model.summary()

# Train the model
history = alexnet_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5 # reduced epochs for testing purposes, increase to 10 to get better results.
)

# evaluate the model on the validation set
val_loss, val_acc = alexnet_model.evaluate(validation_dataset, verbose=2)
print(f"Validation accuracy: {val_acc:.4f}")

# plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy for AlexNet')
plt.show()


Testing different batch sizes

In [ ]:
# Install necessary libraries
!pip install patool==1.12

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import patoolib
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Extract the zip file from Google Drive
#patoolib.extract_archive("/content/drive/MyDrive/Hood College/archive.zip", outdir="/content/extracted_images")

# Define the directory containing the dataset
data_dir = '/content/extracted_images/leapGestRecog'

# Function to load dataset with a specified batch size
def load_dataset(batch_size):
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="training",
        seed=123,
        image_size=(227, 227),
        batch_size=batch_size,
        label_mode='categorical'
    )

    validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="validation",
        seed=123,
        image_size=(227, 227),
        batch_size=batch_size,
        label_mode='categorical'
    )
    return train_dataset, validation_dataset

# Build AlexNet model with Batch Normalization applied for more consistant results
def build_alexnet_with_bn(input_shape=(227, 227, 3), num_classes=10):
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define a function to train the model with different batch sizes
def train_with_batch_size(batch_size):
    print(f"\nTraining with Batch Size = {batch_size}\n")

    # load the dataset with the given batch size
    train_dataset, validation_dataset = load_dataset(batch_size)


    alexnet_model = build_alexnet_with_bn()

    # compile the model
    alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # train the model
    history = alexnet_model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=5 # reduced to 5 for testing purpose increase t o10 for better results
    )

    #evaluate the model on the validation set
    val_loss, val_acc = alexnet_model.evaluate(validation_dataset, verbose=2)
    print(f"Validation accuracy for Batch Size = {batch_size}: {val_acc:.4f}")

    # Plot the training and validation accuracy
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'Training and Validation Accuracy for Batch Size = {batch_size}')
    plt.show()

# Test different batch sizes and gather results
batch_sizes = [16, 32, 64, 128]

for batch_size in batch_sizes:
    train_with_batch_size(batch_size)


Testing with data augmentation

In [ ]:
# Install necessary libraries
!pip install patool==1.12

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import patoolib
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Extract the zip file from Google Drive
# patoolib.extract_archive("/content/drive/MyDrive/Hood College/archive.zip", outdir="/content/extracted_images")

# Define the directory containing the dataset
data_dir = '/content/extracted_images/leapGestRecog'

# Function for loading dataset with data augmentation adjustments
def load_dataset_with_augmentation(batch_size):
    datagen = ImageDataGenerator(
        validation_split=0.3,
        rotation_range=20,  # rotate images up to 20 degrees
        horizontal_flip=True,  # flip images horizontally
        zoom_range=0.2  #scale up by 20%
    )

    train_dataset = datagen.flow_from_directory(
        data_dir,
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',  # set as training data
        seed=123
    )

    validation_dataset = datagen.flow_from_directory(
        data_dir,
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',  # set as validation data
        seed=123
    )
    return train_dataset, validation_dataset

# build AlexNet model
def build_alexnet_with_bn(input_shape=(227, 227, 3), num_classes=10):
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define a function to train the model with data augmentation
def train_with_data_augmentation(batch_size):
    print(f"\nTraining with Batch Size = {batch_size} and Data Augmentation (Flip, Rotate, Scale)\n")


    train_dataset, validation_dataset = load_dataset_with_augmentation(batch_size)

    # create the AlexNet model
    alexnet_model = build_alexnet_with_bn()

    # compile the model
    alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # train the model
    history = alexnet_model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=5  # Adjust the number of epochs as needed (decresed from 10 to 5 for testing)
    )

    #evaluate the model on the validation set
    val_loss, val_acc = alexnet_model.evaluate(validation_dataset, verbose=2)
    print(f"Validation accuracy for Batch Size = {batch_size} with Data Augmentation: {val_acc:.4f}")

    # plot the training and validation accuracy
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'Training and Validation Accuracy for Batch Size = {batch_size} with Data Augmentation')
    plt.show()

# fixed batch size of 32
train_with_data_augmentation(batch_size=32)


This section tests different learning rates for AlexNet [0.1, 0.01, 0.001, 0.0001]

In [ ]:
# Install necessary libraries
!pip install patool==1.12

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import patoolib
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Extract the zip file from Google Drive
#patoolib.extract_archive("/content/drive/MyDrive/Hood College/archive.zip", outdir="/content/extracted_images")

# define the directory containing the dataset
data_dir = '/content/extracted_images/leapGestRecog'

# Function for loading the dataset
def load_dataset(batch_size):
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="training",
        seed=123,
        image_size=(227, 227),
        batch_size=batch_size,
        label_mode='categorical'
    )

    validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="validation",
        seed=123,
        image_size=(227, 227),
        batch_size=batch_size,
        label_mode='categorical'
    )
    return train_dataset, validation_dataset

# Build AlexNet model
def build_alexnet_with_bn(input_shape=(227, 227, 3), num_classes=10):
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# function to train the model with different learning rates
def train_with_learning_rates(batch_size, learning_rates):
    for lr in learning_rates:
        print(f"\nTraining with Learning Rate = {lr}\n")

        # Load the dataset with the given batch size
        train_dataset, validation_dataset = load_dataset(batch_size)

        alexnet_model = build_alexnet_with_bn()

        # compile the model with different learning rates
        optimizer = optimizers.Adam(learning_rate=lr)
        alexnet_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # train the model
        history = alexnet_model.fit(
            train_dataset,
            validation_data=validation_dataset,
            epochs=5  # adjust the number of epochs as needed (fom 10 to 5 for testing purpose)
        )

        # evaluate the model on the validation set
        val_loss, val_acc = alexnet_model.evaluate(validation_dataset, verbose=2)
        print(f"Validation accuracy for Learning Rate = {lr}: {val_acc:.4f}")

        #plot the training and validation accuracy
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='lower right')
        plt.title(f'Training and Validation Accuracy for Learning Rate = {lr}')
        plt.show()

# testing with a fixed batch size and different learning rates
learning_rates = [0.1, 0.01, 0.001, 0.0001]
train_with_learning_rates(batch_size=32, learning_rates=learning_rates)


this section will test the RMSprop optimizer the adam optimier has already been applied through testing the seperate model.

In [ ]:
# Install necessary libraries
!pip install patool==1.12

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import patoolib
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Extract the zip file from Google Drive
#patoolib.extract_archive("/content/drive/MyDrive/Hood College/archive.zip", outdir="/content/extracted_images")

# Define the directory containing the dataset
data_dir = '/content/extracted_images/leapGestRecog'

# Function for loading the dataset
def load_dataset(batch_size):
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="training",
        seed=123,
        image_size=(227, 227),
        batch_size=batch_size,
        label_mode='categorical'
    )

    validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="validation",
        seed=123,
        image_size=(227, 227),
        batch_size=batch_size,
        label_mode='categorical'
    )
    return train_dataset, validation_dataset

# build AlexNet model
def build_alexnet_with_bn(input_shape=(227, 227, 3), num_classes=10):
    model = models.Sequential([
        layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to train the model with RMSprop optimizer
def train_with_rmsprop(batch_size, learning_rate=0.001):
    print(f"\nTraining with RMSprop Optimizer and Learning Rate = {learning_rate}\n")

    # Load the dataset with the given batch size
    train_dataset, validation_dataset = load_dataset(batch_size)

    # Create the AlexNet
    alexnet_model = build_alexnet_with_bn()

    # compile the model with RMSprop optimizer
    optimizer = optimizers.RMSprop(learning_rate=learning_rate)
    alexnet_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # train the model
    history = alexnet_model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=5
    )

    # Evaluate the model on the validation set
    val_loss, val_acc = alexnet_model.evaluate(validation_dataset, verbose=2)
    print(f"Validation accuracy: {val_acc:.4f}")

    # Plot the training and validation accuracy
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'Training and Validation Accuracy with RMSprop Optimizer')
    plt.show()

# Test with a fixed batch size and RMSprop optimizer
train_with_rmsprop(batch_size=32, learning_rate=0.001)
